In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,521 kB]
Hit:14 http://ppa.launchp

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

## Use Pyspark to perform the vine_review analysis


In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FU16928EP5TC|          5|            0|          0|   N|                Y|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|                Y|
| R52R85WC6TIAH|          4|           17|         18|   N|                Y|
| R7HOOYTVIB0DS|          5|            0|          0|   N|                Y|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
helpful_review = vine_df.filter(vine_df.total_votes > "20")
helpful_review.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1CY3APKBOVUYU|          1|            9|         33|   N|                Y|
|R2LTI28AN2281A|          1|            0|         22|   N|                Y|
|R3PENXZQNYN29L|          1|            1|         26|   N|                Y|
|R2P8B83Q61L4HS|          1|            0|         22|   N|                Y|
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R1TIDPK91S2IRQ|          1|            0|         25|   N|                Y|
|R1WQ2C3EYOLKKH|          1|            1|         24|   N|                Y|
|R1H9AEOAFUK08M|          1|            4|         23|   N|     

In [13]:
# create helpful_rate df

helpful_rate_df = helpful_review.filter(helpful_review.helpful_votes/helpful_review.total_votes>="0.50")
helpful_rate_df.show(30)
helpful_rate_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|
|R2ISFKK57JF02V|          5|           25|         26|   N|                Y|
|R3D8LNOI6ZPOFY|          5|           68|         72|   N|                Y|
| RVR6HYB370XXD|          4|          110|        112|   N|     

10517

In [12]:
# Create Vine Program DataFrame for reviews that are a part of the program and whose rating were paid
vine_yes_df = helpful_rate_df.filter(helpful_rate_df.vine == "Y")
vine_yes_df.show()
vine_yes_df.count()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



0

In [11]:
# Create Vine Program DataFrame for reviews that are a part of the program and whose rating were paid
vine_no_df = helpful_rate_df.filter(helpful_rate_df.vine == "N")
vine_no_df.show()
vine_no_df.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|
|R2ISFKK57JF02V|          5|           25|         26|   N|                Y|
|R3D8LNOI6ZPOFY|          5|           68|         72|   N|                Y|
| RVR6HYB370XXD|          4|          110|        112|   N|     

10517

In [14]:
#Number of 5 stars review
five_stars_review = helpful_review.filter(helpful_review.star_rating == "5")
five_stars_review.count()

4524

In [15]:
# Total number of unpaid 5 stars reviews
unpaid_five_stars_review = vine_no_df.filter(vine_no_df.star_rating == "5")
unpaid_five_stars_review.count()

4373

In [16]:
# Total number of paid 5 stars reviews
paid_five_stars_review = vine_yes_df.filter(vine_no_df.star_rating == "5")
paid_five_stars_review.count()

0

In [23]:
# Percentage of unpaid 5 stars reviews
unpaid_5star_review_percent = unpaid_five_stars_review.count() / vine_no_df.count() *100

print(str(round(unpaid_5star_review_percent,2))+ "%")

41.58%


In [24]:
paid_5star_review_percent = paid_five_stars_review.count() / vine_no_df.count() *100

print(str(round(paid_5star_review_percent,2))+ "%")

0.0%
